In [1]:
from pyspark.sql import SparkSession


database = "hacking"
collection = "null_stuff"
connectionString = "mongodb://127.0.0.1"

spark = SparkSession \
    .builder \
    .appName("missing_dataframe") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5') \
    .config("spark.mongodb.input.uri", connectionString) \
    .config("spark.mongodb.output.uri", connectionString) \
    .getOrCreate()


22/11/10 13:40:41 WARN Utils: Your hostname, mbp.local resolves to a loopback address: 127.0.0.1; using 192.168.18.6 instead (on interface en0)
22/11/10 13:40:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/tmillar/dev/repo/hacking-spark-with-mongo/venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tmillar/.ivy2/cache
The jars for the packages stored in: /Users/tmillar/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fdf5424b-15d3-44fa-a35b-da1d4e1a17f4;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.5 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 2098ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.5 from central in [default]
	---------------------------------------------------------------------
	|                 

22/11/10 13:40:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/10 13:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/10 13:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/10 13:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/10 13:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/10 13:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/11/10 13:40:45 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [2]:
df = spark.read.format("mongodb") \
    .option("database", database) \
    .option("collection", collection) \
    .load()

In [3]:
df_csv = spark.read.csv("./../sample_data/null.csv", inferSchema=True, header=True)

In [4]:
df_csv.show()

+----------+-----+-----+
|EmployeeId| Name|Sales|
+----------+-----+-----+
|      emp1| John| null|
|      emp2| null| null|
|      emp3| null|345.0|
|      emp4|Cindy|456.0|
+----------+-----+-----+



Write nulls Dataframe from csv into Mongo

In [5]:
df_csv.write.format("mongodb") \
    .option("collection", collection) \
    .option("database", database) \
    .mode("overwrite") \
    .save()


Show what was written

In [6]:
df_csv.show()

+----------+-----+-----+
|EmployeeId| Name|Sales|
+----------+-----+-----+
|      emp1| John| null|
|      emp2| null| null|
|      emp3| null|345.0|
|      emp4|Cindy|456.0|
+----------+-----+-----+



Read stock collection from Mongo

In [7]:
df = spark.read.format("mongodb") \
    .option("collection", collection) \
    .option("database", database) \
    .load()


In [8]:
df.show()

+----------+-----+-----+--------------------+
|EmployeeId| Name|Sales|                 _id|
+----------+-----+-----+--------------------+
|      emp1| John| null|636d61d305fc0570f...|
|      emp2| null| null|636d61d305fc0570f...|
|      emp3| null|345.0|636d61d305fc0570f...|
|      emp4|Cindy|456.0|636d61d305fc0570f...|
+----------+-----+-----+--------------------+



Drop rows with any null values

In [ ]:
df.na.drop().show()


Drop rows with two or more null values

In [ ]:
df.na.drop(thresh=2).show()

Drop rows using `how`
- drop if any values in row are null
- drop if all values in row are null

In [ ]:
df.na.drop(how="any").show()
df.na.drop(how="all").show()

Drop rows using `subset` which drops row if a column is null

In [ ]:
df.na.drop(subset=["Sales"]).show()
df.na.drop(subset=["Name"]).show()
df.na.drop(subset=["Name", "Sales"]).show()

Fill rows using `fill` which takes the argument and will fill nulls by data type

- subset allows selecting just columns to fill

In [ ]:
df.na.fill("FILL STRING COLUMN").show()
df.na.fill(0).show()
df.na.fill(0, subset=["Sales"]).show()

Fill rows using `fill` but use the `mean` value for sales

In [19]:
from pyspark.sql.functions import mean

mean_rows = df.select(mean("Sales")).collect()
mean_val = mean_rows[0][0]

df.na.fill(mean_val, subset=["Sales"]).show()


+----------+-----+-----+--------------------+
|EmployeeId| Name|Sales|                 _id|
+----------+-----+-----+--------------------+
|      emp1| John|400.5|636d61d305fc0570f...|
|      emp2| null|400.5|636d61d305fc0570f...|
|      emp3| null|345.0|636d61d305fc0570f...|
|      emp4|Cindy|456.0|636d61d305fc0570f...|
+----------+-----+-----+--------------------+

